In [6]:
import pandas as pd
# 讀取 CSV 檔案到資料框中
df = pd.read_csv('scraping_data/0531_no_label_data.csv')
# 顯示資料框的前幾行
df

,stock_symbol,title,date,paragraphs
0,COST,Costco Wholesale (COST) Q3 2024 Earnings Call ...,"May 30, 2024",rator\n\nThank you for standing by. My name is...
1,MRVL,Marvell Technology (MRVL) Q1 2025 Earnings Cal...,"May 30, 2024","perator\n\nGood afternoon, and welcome to Marv..."
2,GPS,Gap (GPS) Q1 2024 Earnings Call Transcript,"May 30, 2024","ternoon, ladies and gentlemen. I would like to..."
3,DG,Dollar General (DG) Q1 2024 Earnings Call Tran...,"May 30, 2024","tor\n\nGood morning. My name is Donna, and I w..."
4,DLTH,Duluth (DLTH) Q1 2024 Earnings Call Transcript,"May 30, 2024","od morning, everyone, and welcome to the Dulut..."
...,...,...,...,...
6373,AXTI,AXT (AXTI) Q4 2021 Earnings Call Transcript,"Feb 16, 2022","fternoon, everyone, and welcome to AXT's fourt..."
6374,BCOV,Brightcove (BCOV) Q4 2021 Earnings Call Transc...,"Feb 16, 2022",tings. Welcome to the Brightcove fourth quarte...
6375,OCDX,Ortho Clinical Diagnostics Holdings plc (OCDX)...,"Feb 16, 2022",",5:00 p.m. ET\n\nOperator\n\nWelcome to Ortho ..."
6376,APP,AppLovin Corporation (APP) Q4 2021 Earnings Ca...,"Feb 16, 2022",\nOperator\n\nGreetings. Welcome to AppLovin C...


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# 設定 DataFrame 的欄位名稱
columns = ['stock_symbol', 'title', 'date', 'paragraphs'
           '0_day_close', '1_day_close', '2_day_close', '3_day_close', 
           '4_day_close', '5_day_close', '7_day_close', '14_day_close', 
           '1_month_close', '2_month_close']

# 初始化新的 DataFrame
new_df = pd.DataFrame(columns=columns)

# 自定義函數來抓取股價資料並計算指定日期的收盤價
def get_close_prices(ticker, date):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=date, end=(datetime.strptime(date, "%Y-%m-%d") + timedelta(days=60)).strftime("%Y-%m-%d"))
    
    close_prices = {}
    if date in hist.index:
        close_prices['0_day_close'] = hist.loc[date]['Close']
        # 檢查每個日期是否存在於資料中，否則返回 None
        close_prices['1_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 1]]['Close'] if (hist.index.get_loc(date) + 1) < len(hist.index) else None
        close_prices['2_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 2]]['Close'] if (hist.index.get_loc(date) + 2) < len(hist.index) else None
        close_prices['3_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 3]]['Close'] if (hist.index.get_loc(date) + 3) < len(hist.index) else None
        close_prices['4_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 4]]['Close'] if (hist.index.get_loc(date) + 4) < len(hist.index) else None
        close_prices['5_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 5]]['Close'] if (hist.index.get_loc(date) + 5) < len(hist.index) else None
        close_prices['7_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 7]]['Close'] if (hist.index.get_loc(date) + 7) < len(hist.index) else None
        close_prices['14_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 14]]['Close'] if (hist.index.get_loc(date) + 14) < len(hist.index) else None
        
        # 計算一個月和兩個月後的日期
        one_month_date = (datetime.strptime(date, "%Y-%m-%d") + timedelta(days=30)).strftime("%Y-%m-%d")
        two_month_date = (datetime.strptime(date, "%Y-%m-%d") + timedelta(days=60)).strftime("%Y-%m-%d")
        close_prices['1_month_close'] = hist.loc[one_month_date]['Close'] if one_month_date in hist.index else None
        close_prices['2_month_close'] = hist.loc[two_month_date]['Close'] if two_month_date in hist.index else None
    else:
        close_prices = {key: None for key in ['0_day_close', '1_day_close', '2_day_close', '3_day_close', 
                                               '4_day_close', '5_day_close', '7_day_close', '14_day_close', 
                                               '1_month_close', '2_month_close']}
    return close_prices

# 遍歷每一行並計算收盤價
for index, row in df.iterrows():
    date_str = row['date']
    ticker_symbol = row['stock_symbol']
    title = row['title']
    paragraphs = row['paragraphs']
    close_prices = get_close_prices(ticker_symbol, date_str)
    
    # 添加到新 DataFrame
    new_row = pd.DataFrame([{
        'stock_symbol': ticker_symbol,
        'title': title,
        'date': date_str,
        'paragraphs': paragraphs,
        **close_prices
    }])
    new_df = pd.concat([new_df, new_row], ignore_index=True)

# 顯示新 DataFrame
new_df

C:\Users\User\AppData\Local\Temp\ipykernel_14708\2206325152.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, new_row], ignore_index=True)
TRMR: No timezone found, symbol may be delisted
C:\Users\User\AppData\Local\Temp\ipykernel_14708\2206325152.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, new_row], ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_14708\2206325152.py:58: FutureWarning: The behavior of DataFrame conc

,stock_symbol,title,date,paragraphs0_day_close,1_day_close,2_day_close,3_day_close,4_day_close,5_day_close,7_day_close,14_day_close,1_month_close,2_month_close,paragraphs,0_day_close
0,XPEV,XPeng (XPEV) Q1 2024 Earnings Call Transcript,2024-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"o, ladies and gentlemen. Thank you for standin...",8.770000
1,LOW,Lowe's Companies (LOW) Q1 2024 Earnings Call T...,2024-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"rator\n\nGood morning, everyone. Welcome to Lo...",224.860001
2,M,Macy's (M) Q1 2024 Earnings Call Transcript,2024-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"ngs, and welcome to the Macy's, Inc. first-qua...",20.080000
3,YMM,Full Truck Alliance (YMM) Q1 2024 Earnings Cal...,2024-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Operator\n\nLadies and gentlemen, good day and...",9.240000
4,ZM,Zoom Video Communications (ZM) Q1 2025 Earning...,2024-05-20,NaN,63.860001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"ET\n\nKelcey McKinley\n\nHello, everyone, and...",64.089996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3907,UNM,Unum Group (UNM) Q2 2022 Earnings Call Transcript,2022-08-03,NaN,33.363258,34.026730,34.055164,34.870293,35.363155,36.860710,37.021839,35.666458,None,"Hello, everyone, and welcome to the Unum Group...",34.055164
3908,ALE,ALLETE (ALE) Q2 2022 Earnings Call Transcript,2022-08-03,NaN,56.870018,55.820759,55.939373,56.724033,57.070747,58.374393,56.106174,55.377758,None,"od day, and welcome to the ALLETE second quart...",57.508698
3909,CVS,CVS Health (CVS) Q2 2022 Earnings Call Transcript,2022-08-03,NaN,96.276520,96.645111,96.399384,97.996590,98.336815,100.548325,95.577148,93.979942,None,"\nLadies and gentlemen, good morning, and welc...",95.813423
3910,LAMR,Lamar Advertising (LAMR) Q2 2022 Earnings Call...,2022-08-03,NaN,91.348869,91.850952,93.594498,93.375412,95.365417,96.351295,89.276711,84.429489,None,"perator\n\nExcuse me, everyone. We now have Se...",91.394516


In [5]:
new_df

,stock_symbol,title,date,paragraphs0_day_close,1_day_close,2_day_close,3_day_close,4_day_close,5_day_close,7_day_close,14_day_close,1_month_close,2_month_close,paragraphs,0_day_close
0,XPEV,XPeng (XPEV) Q1 2024 Earnings Call Transcript,2024-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"o, ladies and gentlemen. Thank you for standin...",8.770000
1,LOW,Lowe's Companies (LOW) Q1 2024 Earnings Call T...,2024-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"rator\n\nGood morning, everyone. Welcome to Lo...",224.860001
2,M,Macy's (M) Q1 2024 Earnings Call Transcript,2024-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"ngs, and welcome to the Macy's, Inc. first-qua...",20.080000
3,YMM,Full Truck Alliance (YMM) Q1 2024 Earnings Cal...,2024-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Operator\n\nLadies and gentlemen, good day and...",9.240000
4,ZM,Zoom Video Communications (ZM) Q1 2025 Earning...,2024-05-20,NaN,63.860001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"ET\n\nKelcey McKinley\n\nHello, everyone, and...",64.089996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3907,UNM,Unum Group (UNM) Q2 2022 Earnings Call Transcript,2022-08-03,NaN,33.363258,34.026730,34.055164,34.870293,35.363155,36.860710,37.021839,35.666458,None,"Hello, everyone, and welcome to the Unum Group...",34.055164
3908,ALE,ALLETE (ALE) Q2 2022 Earnings Call Transcript,2022-08-03,NaN,56.870018,55.820759,55.939373,56.724033,57.070747,58.374393,56.106174,55.377758,None,"od day, and welcome to the ALLETE second quart...",57.508698
3909,CVS,CVS Health (CVS) Q2 2022 Earnings Call Transcript,2022-08-03,NaN,96.276520,96.645111,96.399384,97.996590,98.336815,100.548325,95.577148,93.979942,None,"\nLadies and gentlemen, good morning, and welc...",95.813423
3910,LAMR,Lamar Advertising (LAMR) Q2 2022 Earnings Call...,2022-08-03,NaN,91.348869,91.850952,93.594498,93.375412,95.365417,96.351295,89.276711,84.429489,None,"perator\n\nExcuse me, everyone. We now have Se...",91.394516


In [6]:
new_df.to_csv('0522_price_volatility.csv', index=False)

In [12]:
# 新增 後一天change rate
new_df['1_day_change_rate'] = (new_df['1_day_close'] - new_df['0_day_close']) / new_df['0_day_close']

# add label
new_df['1_day_change_label_binary'] = new_df['1_day_change_rate'].apply(lambda x: 1 if x > 0 else -1)

# 顯示資料框的前幾行，包括新欄位
result = new_df[['stock_symbol', 'title', 'date', 'paragraphs', '1_day_change_rate', 'label']]
result

,stock_symbol,title,date,paragraphs,1_day_change_rate,label
0,XPEV,XPeng (XPEV) Q1 2024 Earnings Call Transcript,2024-05-21,"o, ladies and gentlemen. Thank you for standin...",NaN,-1
1,LOW,Lowe's Companies (LOW) Q1 2024 Earnings Call T...,2024-05-21,"rator\n\nGood morning, everyone. Welcome to Lo...",NaN,-1
2,M,Macy's (M) Q1 2024 Earnings Call Transcript,2024-05-21,"ngs, and welcome to the Macy's, Inc. first-qua...",NaN,-1
3,YMM,Full Truck Alliance (YMM) Q1 2024 Earnings Cal...,2024-05-21,"Operator\n\nLadies and gentlemen, good day and...",NaN,-1
4,ZM,Zoom Video Communications (ZM) Q1 2025 Earning...,2024-05-20,"ET\n\nKelcey McKinley\n\nHello, everyone, and...",-0.003589,-1
...,...,...,...,...,...,...
3907,UNM,Unum Group (UNM) Q2 2022 Earnings Call Transcript,2022-08-03,"Hello, everyone, and welcome to the Unum Group...",-0.020317,-1
3908,ALE,ALLETE (ALE) Q2 2022 Earnings Call Transcript,2022-08-03,"od day, and welcome to the ALLETE second quart...",-0.011106,-1
3909,CVS,CVS Health (CVS) Q2 2022 Earnings Call Transcript,2022-08-03,"\nLadies and gentlemen, good morning, and welc...",0.004833,1
3910,LAMR,Lamar Advertising (LAMR) Q2 2022 Earnings Call...,2022-08-03,"perator\n\nExcuse me, everyone. We now have Se...",-0.000499,-1


In [13]:
result = result.dropna(subset=['1_day_change_rate'])
result

,stock_symbol,title,date,paragraphs,1_day_change_rate,label
4,ZM,Zoom Video Communications (ZM) Q1 2025 Earning...,2024-05-20,"ET\n\nKelcey McKinley\n\nHello, everyone, and...",-0.003589,-1
5,PANW,Palo Alto Networks (PANW) Q3 2024 Earnings Cal...,2024-05-20,"Walter Pritchard\n\nGood day, everyone, and we...",-0.037403,-1
7,WIX,Wix.com (WIX) Q1 2024 Earnings Call Transcript,2024-05-20,"d day, and thank you for standing by. Welcome ...",0.029639,1
8,LI,Li Auto (LI) Q1 2024 Earnings Call Transcript,2024-05-20,"lo, ladies and gentlemen. Thank you for standi...",-0.034546,-1
9,GLBE,Global-e Online (GLBE) Q1 2024 Earnings Call T...,2024-05-20,rator\n\nWelcome to Global-e first quarter 202...,-0.021180,-1
...,...,...,...,...,...,...
3907,UNM,Unum Group (UNM) Q2 2022 Earnings Call Transcript,2022-08-03,"Hello, everyone, and welcome to the Unum Group...",-0.020317,-1
3908,ALE,ALLETE (ALE) Q2 2022 Earnings Call Transcript,2022-08-03,"od day, and welcome to the ALLETE second quart...",-0.011106,-1
3909,CVS,CVS Health (CVS) Q2 2022 Earnings Call Transcript,2022-08-03,"\nLadies and gentlemen, good morning, and welc...",0.004833,1
3910,LAMR,Lamar Advertising (LAMR) Q2 2022 Earnings Call...,2022-08-03,"perator\n\nExcuse me, everyone. We now have Se...",-0.000499,-1


In [14]:
result = result.reset_index(drop=True)
result

,stock_symbol,title,date,paragraphs,1_day_change_rate,label
0,ZM,Zoom Video Communications (ZM) Q1 2025 Earning...,2024-05-20,"ET\n\nKelcey McKinley\n\nHello, everyone, and...",-0.003589,-1
1,PANW,Palo Alto Networks (PANW) Q3 2024 Earnings Cal...,2024-05-20,"Walter Pritchard\n\nGood day, everyone, and we...",-0.037403,-1
2,WIX,Wix.com (WIX) Q1 2024 Earnings Call Transcript,2024-05-20,"d day, and thank you for standing by. Welcome ...",0.029639,1
3,LI,Li Auto (LI) Q1 2024 Earnings Call Transcript,2024-05-20,"lo, ladies and gentlemen. Thank you for standi...",-0.034546,-1
4,GLBE,Global-e Online (GLBE) Q1 2024 Earnings Call T...,2024-05-20,rator\n\nWelcome to Global-e first quarter 202...,-0.021180,-1
...,...,...,...,...,...,...
3755,UNM,Unum Group (UNM) Q2 2022 Earnings Call Transcript,2022-08-03,"Hello, everyone, and welcome to the Unum Group...",-0.020317,-1
3756,ALE,ALLETE (ALE) Q2 2022 Earnings Call Transcript,2022-08-03,"od day, and welcome to the ALLETE second quart...",-0.011106,-1
3757,CVS,CVS Health (CVS) Q2 2022 Earnings Call Transcript,2022-08-03,"\nLadies and gentlemen, good morning, and welc...",0.004833,1
3758,LAMR,Lamar Advertising (LAMR) Q2 2022 Earnings Call...,2022-08-03,"perator\n\nExcuse me, everyone. We now have Se...",-0.000499,-1


In [15]:
result = result[(result != 0).any(axis=1)]
result

,stock_symbol,title,date,paragraphs,1_day_change_rate,label
0,ZM,Zoom Video Communications (ZM) Q1 2025 Earning...,2024-05-20,"ET\n\nKelcey McKinley\n\nHello, everyone, and...",-0.003589,-1
1,PANW,Palo Alto Networks (PANW) Q3 2024 Earnings Cal...,2024-05-20,"Walter Pritchard\n\nGood day, everyone, and we...",-0.037403,-1
2,WIX,Wix.com (WIX) Q1 2024 Earnings Call Transcript,2024-05-20,"d day, and thank you for standing by. Welcome ...",0.029639,1
3,LI,Li Auto (LI) Q1 2024 Earnings Call Transcript,2024-05-20,"lo, ladies and gentlemen. Thank you for standi...",-0.034546,-1
4,GLBE,Global-e Online (GLBE) Q1 2024 Earnings Call T...,2024-05-20,rator\n\nWelcome to Global-e first quarter 202...,-0.021180,-1
...,...,...,...,...,...,...
3755,UNM,Unum Group (UNM) Q2 2022 Earnings Call Transcript,2022-08-03,"Hello, everyone, and welcome to the Unum Group...",-0.020317,-1
3756,ALE,ALLETE (ALE) Q2 2022 Earnings Call Transcript,2022-08-03,"od day, and welcome to the ALLETE second quart...",-0.011106,-1
3757,CVS,CVS Health (CVS) Q2 2022 Earnings Call Transcript,2022-08-03,"\nLadies and gentlemen, good morning, and welc...",0.004833,1
3758,LAMR,Lamar Advertising (LAMR) Q2 2022 Earnings Call...,2022-08-03,"perator\n\nExcuse me, everyone. We now have Se...",-0.000499,-1


In [3]:
df_selected.to_csv('0522_no_label_data.csv', index=False)